In [1]:
import struct
import zlib

f = open('example.png', 'rb')

In [2]:
PngSignature = b'\x89PNG\r\n\x1a\n'
if f.read(len(PngSignature)) != PngSignature:
    raise Exception('Invalid PNG Signature')

In [3]:
def read_chunk(f):
    # Returns (chunk_type, chunk_data)
    chunk_length, chunk_type = struct.unpack('>I4s', f.read(8))
    chunk_data = f.read(chunk_length)
    checksum = zlib.crc32(chunk_data, zlib.crc32(struct.pack('>4s', chunk_type)))
    chunk_crc, = struct.unpack('>I', f.read(4))
    if chunk_crc != checksum:
        raise Exception('chunk checksum failed {} != {}'.format(chunk_crc,
            checksum))
    return chunk_type, chunk_data

In [4]:
chunks = []
while True:
    chunk_type, chunk_data = read_chunk(f)
    chunks.append((chunk_type, chunk_data))
    if chunk_type == b'IEND':
        break

In [5]:
print([chunk_type for chunk_type, chunk_data in chunks])

[b'IHDR', b'sRGB', b'IDAT', b'IEND']


In [6]:
_, IHDR_data = chunks[0] # IHDR is always first chunk
width, height, bitd, colort, compm, filterm, interlacem = struct.unpack('>IIBBBBB', IHDR_data)

In [13]:
width, height

(233, 70)

In [7]:
IDAT_data = b''.join(chunk_data for chunk_type, chunk_data in chunks if chunk_type == b'IDAT')
IDAT_data

b'x\x9c\xed\x9dw\\SW\x1b\xc7\x9f\x10fX\xe2@\x91\x19D\x01\x192\x15D\x05\xda\xa2Xw](\x14\x17\xa2\xd5*\xb6\xb5\xbe~l\xdf\xb7\xed[\xeb\xa8\xa3\xf5U\xebVTp\x81{\xe2D\x84\x80\x82"C\x99\xb2E\x90\x91\x9d\x90q\xef}\xff\x08M\xa9\x04\x8c!\x10o=\xdf\x0f\x7f\xdc{\x9e\xe7\x9c\xe79\xc9/g\xe4\xdc|\xa0\x10\x04\x01\x08\x04\t\xd1\xd2t\x02\x08\x84\x8a \xed"\xc8\n\xd2.\x82\xac \xed"\xc8\n\xd2.\x82\xac \xed"\xc8\n\xd2.\x82\xac \xed"\xc8\n\xd2.\x82\xac \xed"\xc8\n\xd2.\x82\xac \xed"\xc8\n\xd2.\x82\xac \xed"\xc8\n\xd2.\x82\xac \xed"\xc8\n\xd2.\x82\xac \xed"\xc8\x8a\xb6\xa6\x13@\xfc\x8d\xb8\xf8S\x17/_\xab\xac\xac\x1adO_\xfd\xf5JooO\xe5\xebfe=\xf9u\xdb\x0e\n\x00\x8b\xc5\x9e1}j\xd4\xa2y\x14\n\xa5;\x93\xd54\x04\xe2\xbd\xe1\xe4\xa9\x84]\xbb\xf7\x11\x04\xd1\xd2\xd2\xb2h\xf1r\xc7\xa1^y\xf9\xcf\x95\xac\xdb\xd8\xd84\xc2?\xb8\xb8\xa4\x94 \x88\xd4\xb4t\xba\x83\xeb\xfe\x03G\xba9_\r\xd3\xd3k\x86\xaa\xaa\xea\xa4\x9bwz8(\x00\x14\x15\x95\xdcKN\xe9\xf9\xb8\xef\xc4\xa93g\xaf^O\xc20\\OO\xef\xdf\xdf\xad\x11\x8b\xc5\x07\x0e\xc6*Y

In [8]:
IDAT_data = zlib.decompress(IDAT_data)

In [9]:
def PaethPredictor(a, b, c):
    p = a + b - c
    pa = abs(p - a)
    pb = abs(p - b)
    pc = abs(p - c)
    if pa <= pb and pa <= pc:
        Pr = a
    elif pb <= pc:
        Pr = b
    else:
        Pr = c
    return Pr

Recon = []
bytesPerPixel = 4
stride = width * bytesPerPixel

In [10]:
def Recon_a(r, c):
    return Recon[r * stride + c - bytesPerPixel] if c >= bytesPerPixel else 0

def Recon_b(r, c):
    return Recon[(r-1) * stride + c] if r > 0 else 0

def Recon_c(r, c):
    return Recon[(r-1) * stride + c - bytesPerPixel] if r > 0 and c >= bytesPerPixel else 0

In [12]:
i = 0
for r in range(height): # for each scanline
    filter_type = IDAT_data[i] # first byte of scanline is filter type
    i += 1
    for c in range(stride): # for each byte in scanline
        Filt_x = IDAT_data[i]
        i += 1
        if filter_type == 0: # None
            Recon_x = Filt_x
        elif filter_type == 1: # Sub
            Recon_x = Filt_x + Recon_a(r, c)
        elif filter_type == 2: # Up
            Recon_x = Filt_x + Recon_b(r, c)
        elif filter_type == 3: # Average
            Recon_x = Filt_x + (Recon_a(r, c) + Recon_b(r, c)) // 2
        elif filter_type == 4: # Paeth
            Recon_x = Filt_x + PaethPredictor(Recon_a(r, c), Recon_b(r, c), Recon_c(r, c))
        # else:
        #     raise Exception('unknown filter type: ' + str(filter_type))
        Recon.append(Recon_x & 0xff) # truncation to byte

IndexError: index out of range

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.imshow(np.array(Recon).reshape((height, width, 4)))
plt.show()

NameError: name 'Recon' is not defined